In [1]:
import os
import sys
import getpass
from binance.spot import Spot

__file__ = %pwd
loc_list = os.path.abspath(__file__).split(os.sep)
HOME_LOC = os.path.join(os.sep, *loc_list[:-2])
sys.path.append(HOME_LOC)
os.chdir(HOME_LOC)
from CODE.Utils.encrypt import *
from CODE.Utils.encrypt import Encrypted_API_key
from CODE.Utils.encrypt import Encrypted_API_secret
from CODE.Utils.utils import *
from CODE.Utils.indicators import *
from CODE.Utils.normalize import *
from CODE.Runner import *
from CODE.RNN import *


/homes/David_Li/Mega/University_of_Adelaide/Works/Courses/4339_COMP_SCI_7318_Deep_Learning_Fundamentals/Assignment3
/homes/David_Li/Mega/University_of_Adelaide/Works/Courses/4339_COMP_SCI_7318_Deep_Learning_Fundamentals/Assignment3


In [2]:
file_paths = load_and_sort_files(os.path.join(HOME_LOC, "DATA", "RAW", "1d")
train_X, train_Y, test_X, test_Y = split_data(
    file_paths, train_ratio=0.85, x_length=59, y_length=2
)

Processing Training Files:   0%|          | 0/85 [00:00<?, ?it/s]

Processing Testing Files: 100%|██████████| 15/15 [00:01<00:00, 12.24it/s]


In [3]:
print(train_X.shape, train_Y.shape, test_X.shape, test_Y.shape)

(85, 47, 23) (85, 2) (15, 47, 23) (15, 2)


In [4]:
num_channels = train_X.shape[-1]  # 通道数，对应于特征数
output_size = train_Y.shape[-1]  # 输出尺寸
batch_size = 15  # 批处理大小

# 定义卷积层配置
conv_layers_config = [
    (64, 3, 1),  # 第一个卷积层：64个过滤器，核大小为3，填充为1
    (128, 3, 1),  # 第二个卷积层：128个过滤器，核大小为3，填充为1
    (128, 3, 1),
    (128, 3, 1),
]

# RNN层的参数
hidden_size = 100  # RNN隐藏层大小
num_rnn_layers = 2  # RNN层数量

# 创建模型参数字典
model_p = {
    "conv_layers_config": conv_layers_config,
    "num_channels": num_channels,  # 从train_X.shape[-1]获取
    "output_size": output_size,  # 从train_Y.shape[-1]获取
    "data_len": train_X.shape[1],  # 时间序列长度
    "hidden_size": hidden_size,
    "num_rnn_layers": num_rnn_layers,
}

# 创建 GroupedTrainer 实例
trainer = GroupedTrainer(
    model=FlexibleRNN,  # 使用 FlexibleRNN 模型
    data=(train_X, test_X, train_Y, test_Y),  # 传入训练和测试数据
    batch_size=batch_size,  # 指定批处理大小
    model_p=model_p,  # 传入模型参数
    learning_rate=0.01,  # 学习率
    weight_decay=0.0,  # 权重衰减（正则化参数）
)

# 设置损失函数为均方误差（适用于回归任务）
trainer.criterion = nn.MSELoss()

# 打印模型结构（可选）
print(trainer.model)

FlexibleRNN(
  (conv_layers): ModuleList(
    (0): Conv1d(23, 64, kernel_size=(3,), stride=(1,), padding=(1,))
    (1): Conv1d(64, 128, kernel_size=(3,), stride=(1,), padding=(1,))
    (2-3): 2 x Conv1d(128, 128, kernel_size=(3,), stride=(1,), padding=(1,))
  )
  (rnn): RNN(128, 100, num_layers=2, batch_first=True)
  (fc1): Linear(in_features=200, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=2, bias=True)
)


In [6]:
trainer.train(7200, iter_n=10)

total_loop_n: 150, len(self.group_size): 5
group_index: 0
Epoch: 0 Loss: 0.01, Train Sharpe Ratio: -0.52, Train Annualized Return: 0.29, Test Sharpe Ratio: -0.35, Test Annualized Return: 1.18
group_index: 1
group_index: 2
Epoch: 100 Loss: 0.00, Train Sharpe Ratio: -0.10, Train Annualized Return: 0.76, Test Sharpe Ratio: 0.15, Test Annualized Return: 21.68
group_index: 3
group_index: 4


0.008721932768821716